In [1]:
import pandas as pd
import spacy

In [2]:
nlp = spacy.load('pt_core_news_sm')

In [3]:
dados_treino = pd.read_csv("data/treino.csv")
dados_treino.head()

,title,text,date,category,subcategory,link
0,"Após polêmica, Marine Le Pen diz que abomina n...",A candidata da direita nacionalista à Presidên...,2017-04-28,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
1,"Macron e Le Pen vão ao 2º turno na França, em ...",O centrista independente Emmanuel Macron e a d...,2017-04-23,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/04/187...
2,"Apesar de larga vitória nas legislativas, Macr...",As eleições legislativas deste domingo (19) na...,2017-06-19,mundo,NaN,http://www1.folha.uol.com.br/mundo/2017/06/189...
3,"Governo antecipa balanço, e Alckmin anuncia qu...",O número de ocorrências de homicídios dolosos ...,2015-07-24,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2015/07...
4,"Após queda em maio, a atividade econômica sobe...","A economia cresceu 0,25% no segundo trimestre,...",2017-08-17,mercado,NaN,http://www1.folha.uol.com.br/mercado/2017/08/1...


In [4]:
texto = "Rio de !!!janeiro$$$$$ é uma cidade maravilhosa"
doc = nlp(texto)

In [5]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino["title"])

In [6]:
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)
        
    if len(tokens_validos) > 2:
        return " ".join(tokens_validos)

In [7]:
trata_textos(doc)

'Rio cidade maravilhosa'

In [8]:
from time import time

# pegar o início da execução (para calcular o tempo levado para o processamento)
t0 = time()

# processar em paralelo e enviar em lotes
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                       batch_size = 1000,
                                                       n_process = -1)]

tf = time() - t0

print(tf/60)

2.18006778160731


In [9]:
titulos_tratados = pd.DataFrame({"titulo":textos_tratados})

titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


In [10]:
from gensim.models import Word2Vec

w2v_modelo = Word2Vec(sg = 0,
                     window = 2,
                     #size = 300,
                     min_count = 5,
                     alpha = 0.03,
                     min_alpha = 0.007)

In [11]:
w2v_modelo

In [12]:
print(len(titulos_tratados))

titulos_tratados = titulos_tratados.dropna().drop_duplicates()

print(len(titulos_tratados))

90000
84466


In [13]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [14]:
import logging

logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

w2v_modelo = Word2Vec(sg = 0,
                     window = 2,
                     #size = 300,
                     min_count = 5,
                     alpha = 0.03,
                     min_alpha = 0.007)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

2021-09-18 18:52:02,265 : - Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=100, alpha=0.03)', 'datetime': '2021-09-18T18:52:02.265406', 'gensim': '4.1.1', 'python': '3.7.12 (default, Sep  8 2021, 01:20:16) \n[GCC 10.2.1 20210110]', 'platform': 'Linux-5.4.72-microsoft-standard-WSL2-x86_64-with-debian-11.0', 'event': 'created'}
2021-09-18 18:52:02,268 : - collecting all words and their counts
2021-09-18 18:52:02,269 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-09-18 18:52:02,299 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2021-09-18 18:52:02,318 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2021-09-18 18:52:02,341 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2021-09-18 18:52:02,360 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2021-09-18 18:52:02,381 : - PROGRESS: at sentence #25000, processed 

In [15]:
w2v_modelo.train(lista_lista_tokens, 
                 total_examples=w2v_modelo.corpus_count,
                 epochs = 30)

2021-09-18 18:52:03,333 : - Word2Vec lifecycle event {'msg': 'training model with 3 workers on 12924 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=2 shrink_windows=True', 'datetime': '2021-09-18T18:52:03.332999', 'gensim': '4.1.1', 'python': '3.7.12 (default, Sep  8 2021, 01:20:16) \n[GCC 10.2.1 20210110]', 'platform': 'Linux-5.4.72-microsoft-standard-WSL2-x86_64-with-debian-11.0', 'event': 'train'}
2021-09-18 18:52:04,214 : - worker thread finished; awaiting finish of 2 more threads
2021-09-18 18:52:04,218 : - worker thread finished; awaiting finish of 1 more threads
2021-09-18 18:52:04,243 : - worker thread finished; awaiting finish of 0 more threads
2021-09-18 18:52:04,245 : - EPOCH - 1 : training on 540242 raw words (486133 effective words) took 0.9s, 547590 effective words/s
2021-09-18 18:52:05,199 : - worker thread finished; awaiting finish of 2 more threads
2021-09-18 18:52:05,221 : - worker thread finished; awaiting finish of 1 more threads
2021-09

2021-09-18 18:52:16,292 : - worker thread finished; awaiting finish of 2 more threads
2021-09-18 18:52:16,301 : - worker thread finished; awaiting finish of 1 more threads
2021-09-18 18:52:16,305 : - worker thread finished; awaiting finish of 0 more threads
2021-09-18 18:52:16,306 : - EPOCH - 21 : training on 540242 raw words (486236 effective words) took 0.5s, 890523 effective words/s
2021-09-18 18:52:16,892 : - worker thread finished; awaiting finish of 2 more threads
2021-09-18 18:52:16,903 : - worker thread finished; awaiting finish of 1 more threads
2021-09-18 18:52:16,910 : - worker thread finished; awaiting finish of 0 more threads
2021-09-18 18:52:16,911 : - EPOCH - 22 : training on 540242 raw words (486072 effective words) took 0.6s, 817504 effective words/s
2021-09-18 18:52:17,445 : - worker thread finished; awaiting finish of 2 more threads
2021-09-18 18:52:17,453 : - worker thread finished; awaiting finish of 1 more threads
2021-09-18 18:52:17,467 : - worker thread finished

(14584205, 16207260)

In [16]:
## Avaliação Qualitativa
w2v_modelo.wv.most_similar("pfizer")

[('tornozeleiras', 0.6402171850204468),
 ('varejistas', 0.6213124990463257),
 ('invepar', 0.6192618012428284),
 ('kraft', 0.6191898584365845),
 ('telefónica', 0.6007035970687866),
 ('lucram', 0.5959634184837341),
 ('leniência', 0.5940981507301331),
 ('cetip', 0.5926719307899475),
 ('negociam', 0.5741658210754395),
 ('braskem', 0.5734907984733582)]

In [17]:
## Análise Qualitativa
w2v_modelo.wv.most_similar("microsoft")

[('unilever', 0.5596420168876648),
 ('sky', 0.5520409345626831),
 ('amazon', 0.5504106283187866),
 ('tesla', 0.5415129065513611),
 ('canais', 0.5408347845077515),
 ('braskem', 0.5335578918457031),
 ('sony', 0.513820230960846),
 ('buffett', 0.494262158870697),
 ('walmart', 0.4938730001449585),
 ('disney', 0.48963168263435364)]

In [18]:
#Treinamento do modelo Skip-gram
# Skip-Gram possui um resultado melhor com um window maior, por isso, alterado de 2 para 5

w2v_modelo_sg = Word2Vec(sg = 1,
                     window = 5,
                     #size = 300,
                     min_count = 5,
                     alpha = 0.03,
                     min_alpha = 0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)

w2v_modelo_sg.train(lista_lista_tokens,
                 total_examples=w2v_modelo_sg.corpus_count,
                 epochs = 30)

2021-09-18 18:52:21,546 : - Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=100, alpha=0.03)', 'datetime': '2021-09-18T18:52:21.546225', 'gensim': '4.1.1', 'python': '3.7.12 (default, Sep  8 2021, 01:20:16) \n[GCC 10.2.1 20210110]', 'platform': 'Linux-5.4.72-microsoft-standard-WSL2-x86_64-with-debian-11.0', 'event': 'created'}
2021-09-18 18:52:21,549 : - collecting all words and their counts
2021-09-18 18:52:21,552 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-09-18 18:52:21,575 : - PROGRESS: at sentence #5000, processed 31930 words, keeping 10193 word types
2021-09-18 18:52:21,605 : - PROGRESS: at sentence #10000, processed 63848 words, keeping 14989 word types
2021-09-18 18:52:21,625 : - PROGRESS: at sentence #15000, processed 95753 words, keeping 18279 word types
2021-09-18 18:52:21,646 : - PROGRESS: at sentence #20000, processed 127689 words, keeping 21033 word types
2021-09-18 18:52:21,666 : - PROGRESS: at sentence #25000, processed 

2021-09-18 18:52:32,820 : - worker thread finished; awaiting finish of 2 more threads
2021-09-18 18:52:32,828 : - worker thread finished; awaiting finish of 1 more threads
2021-09-18 18:52:32,842 : - worker thread finished; awaiting finish of 0 more threads
2021-09-18 18:52:32,844 : - EPOCH - 8 : training on 540242 raw words (486165 effective words) took 1.4s, 357150 effective words/s
2021-09-18 18:52:33,861 : - EPOCH 9 - PROGRESS: at 75.91% examples, 367189 words/s, in_qsize 6, out_qsize 0
2021-09-18 18:52:34,095 : - worker thread finished; awaiting finish of 2 more threads
2021-09-18 18:52:34,146 : - worker thread finished; awaiting finish of 1 more threads
2021-09-18 18:52:34,159 : - worker thread finished; awaiting finish of 0 more threads
2021-09-18 18:52:34,160 : - EPOCH - 9 : training on 540242 raw words (486113 effective words) took 1.3s, 373034 effective words/s
2021-09-18 18:52:35,186 : - EPOCH 10 - PROGRESS: at 77.75% examples, 372676 words/s, in_qsize 5, out_qsize 0
2021-09

2021-09-18 18:52:53,338 : - EPOCH - 24 : training on 540242 raw words (486138 effective words) took 1.2s, 390275 effective words/s
2021-09-18 18:52:54,354 : - EPOCH 25 - PROGRESS: at 81.44% examples, 392915 words/s, in_qsize 5, out_qsize 0
2021-09-18 18:52:54,543 : - worker thread finished; awaiting finish of 2 more threads
2021-09-18 18:52:54,557 : - worker thread finished; awaiting finish of 1 more threads
2021-09-18 18:52:54,560 : - worker thread finished; awaiting finish of 0 more threads
2021-09-18 18:52:54,563 : - EPOCH - 25 : training on 540242 raw words (486147 effective words) took 1.2s, 399723 effective words/s
2021-09-18 18:52:55,581 : - EPOCH 26 - PROGRESS: at 77.75% examples, 374779 words/s, in_qsize 5, out_qsize 0
2021-09-18 18:52:55,779 : - worker thread finished; awaiting finish of 2 more threads
2021-09-18 18:52:55,799 : - worker thread finished; awaiting finish of 1 more threads
2021-09-18 18:52:55,832 : - worker thread finished; awaiting finish of 0 more threads
2021

(14583945, 16207260)

In [19]:
w2v_modelo_sg.wv.most_similar("google")

[('apple', 0.6828131675720215),
 ('facebook', 0.656792402267456),
 ('app', 0.5954843163490295),
 ('chips', 0.5801985263824463),
 ('toshiba', 0.5645671486854553),
 ('amazon', 0.5596395134925842),
 ('yahoo', 0.5568916201591492),
 ('android', 0.5563364624977112),
 ('reguladores', 0.5473930239677429),
 ('waze', 0.5408293604850769)]

# Funções para validação do modelo

In [21]:
import spacy
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors

w2v_modelo_cbow = KeyedVectors.load_word2vec_format("modelos/modelo_cbow.txt")
w2v_modelo_sg = KeyedVectors.load_word2vec_format("modelos/modelo_skipgram.txt")
artigo_treino = pd.read_csv("data/treino.csv")
artigo_teste = pd.read_csv("data/teste.csv")

2021-09-18 19:09:11,126 : - loading projection weights from modelos/modelo_cbow.txt
2021-09-18 19:09:12,767 : - KeyedVectors lifecycle event {'msg': 'loaded (12924, 100) matrix of type float32 from modelos/modelo_cbow.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2021-09-18T19:09:12.767799', 'gensim': '4.1.1', 'python': '3.7.12 (default, Sep  8 2021, 01:20:16) \n[GCC 10.2.1 20210110]', 'platform': 'Linux-5.4.72-microsoft-standard-WSL2-x86_64-with-debian-11.0', 'event': 'load_word2vec_format'}
2021-09-18 19:09:12,769 : - loading projection weights from modelos/modelo_skipgram.txt
2021-09-18 19:09:14,176 : - KeyedVectors lifecycle event {'msg': 'loaded (12924, 100) matrix of type float32 from modelos/modelo_skipgram.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2021-09-18T19:09:14.176913', 'gensim': '4.1.1', 'python': '3.7.12 (default, Sep  8 2021, 01:20:16) \n[GCC 10.2.1 20210110]', 'platform': 'Linux-5.4.72-microsoft-standard-WSL2-x86_64-with-debian-11.0', 'event': 

In [22]:
nlp = spacy.load("pt_core_news_sm", disable=["parser", "ner", "tagger", "textcat"])

def tokenizador(texto):
    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())
    
    return tokens_validos

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
tokens = tokenizador(texto)
print(tokens)

['rio', 'janeiro', 'cidade', 'maravilhosa']


In [26]:
def combinacao_de_vetores_por_soma(palavras, modelo):

    vetor_resultante = np.zeros((1,300))

    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)

        except KeyError:
            pass
                

    return vetor_resultante

print("tokens: ", tokens)
print("w2v_modelo_cbow: ", w2v_modelo_cbow)
#vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
#print(vetor_texto.shape)
#print(vetor_texto)

tokens:  ['rio', 'janeiro', 'cidade', 'maravilhosa']
w2v_modelo_cbow:  <gensim.models.keyedvectors.KeyedVectors object at 0x7f79bf8b7f50>


In [39]:
def matriz_vetores(textos):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))
    
    for i in range(x):
        palavras_numeros = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras_numeros)
        
    return matriz

matriz_vetores_treino = matriz_vetores(artigo_treino.title)
matriz_vetores_teste = matriz_vetores(artigo_treino.title)
print(matriz_vetores_treino)
print(matriz_vetores_teste)

NameError: name 'artigo_treino' is not defined

In [ ]:
def combinacao_vetores_por_soma()

In [35]:
w2v_modelo.wv.save_word2vec_format("modelos/modelo_cbow.txt", binary=False)
w2v_modelo_sg.wv.save_word2vec_format("modelos/modelo_skipgram.txt", binary=False)

2021-09-15 18:41:28,433 : - storing 12924x100 projection weights into modelos/modelo_cbow.txt
2021-09-15 18:41:29,769 : - storing 12924x100 projection weights into modelos/modelo_skipgram.txt


In [38]:
import string
import nltk

def tokenizador(texto):
    texto = texto.lower()
    lista_alfanumerico = []
    
    for token_valido in nltk.word_tokenize(texto):
        if token_valido in string.ponctuation: continue
        lista_alfanumerico.append(token_valido)
            
    return lista_alfanumerico

tokenizador("Texto Exemplo, 1234.")

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/local/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************
